# Menjelaskan Dataset  

## Informasi Atribut Dataset Diabetes

Berikut adalah penjelasan masing-masing atribut pada dataset deteksi diabetes yang digunakan:

- **Pregnancies**  
  Untuk menyatakan jumlah kehamilan.

- **Glucose**  
  Untuk menyatakan kadar glukosa dalam darah.

- **BloodPressure**  
  Untuk menyatakan pengukuran tekanan darah.

- **SkinThickness**  
  Untuk menyatakan ketebalan lipatan kulit.

- **Insulin**  
  Untuk menyatakan kadar insulin dalam darah.

- **BMI**  
  Untuk menyatakan indeks massa tubuh.

- **DiabetesPedigreeFunction**  
  Untuk menyatakan persentase risiko diabetes berdasarkan silsilah keluarga.

- **Age**  
  Untuk menyatakan usia pasien.

- **Outcome**  
  Untuk menyatakan hasil akhir:
  - `1` menunjukkan pasien **positif diabetes (Yes)**.
  - `0` menunjukkan pasien **negatif diabetes (No)**.


# Mempersiapkan Library

## Import Library Untuk Melakukan Prediksi
- `import requests`, `import json`, `import base64`  
  Digunakan untuk:
  - `requests`: melakukan HTTP request jika pipeline akan upload model ke server lain.
  - `json`: parsing dan pembuatan data JSON.
  - `base64`: encoding/decoding data saat komunikasi dengan REST API.

- `import tensorflow as tf`  
  Untuk membuat, melatih, dan menyimpan model deep learning menggunakan TensorFlow.

In [1]:
import requests
import json
import tensorflow as tf
import base64

# # Melakukan Prediksi 

In [2]:
# Membuat dictionary raw_data yang akan digunakan untuk melakukan
# prediksi menggunakan model deteksi diabetes yang telah dilatih
raw_data = {
    "Pregnancies": 6,                   # Jumlah kehamilan
    "Glucose": 148,                     # Kadar glukosa dalam darah
    "BloodPressure": 72,                # Tekanan darah
    "SkinThickness": 35,                # Ketebalan lipatan kulit
    "Insulin": 10,                      # Kadar insulin dalam darah
    "BMI": 33.6,                        # Indeks massa tubuh
    "DiabetesPedigreeFunction": 0.627,  # Persentase risiko diabetes
    "Age": 50                           # Usia pasien
}

In [3]:
# Mendefinisikan fitur mana yang bertipe integer dan float,
# untuk membantu encoding data input ke dalam format TFRecord

int_features = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "Age"]
float_features = ["BMI", "DiabetesPedigreeFunction"]

feature_dict = {}

# Melakukan iterasi pada raw_data untuk mengisi feature_dict sesuai tipe data
for key, value in raw_data.items():
    if key in int_features:
        feature_dict[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    elif key in float_features:
        feature_dict[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    else:
        raise ValueError(f"Unknown feature: {key}")

# Membuat objek tf.train.Example yang berisi semua fitur dalam bentuk serialized
# agar dapat digunakan sebagai input ke model TensorFlow saat prediksi
example = tf.train.Example(features=tf.train.Features(feature=feature_dict))

In [4]:
# Mengubah tf.train.Example menjadi byte string
serialized_example_bytes = example.SerializeToString()

# Mengenkripsi byte string menjadi base64 agar dapat dikirim melalui JSON POST request
encoded_example_string = base64.b64encode(serialized_example_bytes).decode('utf-8')

# Menyusun payload JSON sesuai dengan format TensorFlow Serving REST API
json_data = json.dumps({
    "instances": [{"examples": {"b64": encoded_example_string}}]
})

# Mengirimkan POST request ke endpoint model TFX yang sudah dideploy
response = requests.post(
    'https://mlops2-production-fce5.up.railway.app/v1/models/cc-model:predict',
    data=json_data
)

# Mengambil hasil JSON dari respons server
prediction_output = response.json()

if 'predictions' in prediction_output and len(prediction_output['predictions']) > 0:
    probability = prediction_output['predictions'][0][0]

    # Menentukan threshold klasifikasi untuk diabetes
    threshold = 0.5

    # Melakukan klasifikasi berdasarkan threshold
    if probability >= threshold:
        result = "Diabetes"
    else:
        result = "Bukan Diabetes"

    # Menampilkan hasil probabilitas dan klasifikasi
    print(f"Prediksi probabilitas: {probability}")
    print(f"Hasil klasifikasi: {result}")

else:
    # Jika gagal mendapatkan prediksi, cetak respons untuk debugging
    print("Gagal mendapatkan prediksi atau format respons tidak terduga.")
    print(prediction_output)

Prediksi probabilitas: 0.997304857
Hasil klasifikasi: Diabetes
